In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time # 사이트 기동될 때 까지 2초 정도를 줌. - 셀리니움에서 get. request에서 get쓸 때
import pymysql

In [2]:
# 사이트 기동
def dongaUrl(search):
        url_temp = 'https://www.donga.com/news/search?&query={search}&check_news=1&more=1&sorting=1&search_date=5&v1=20190401&v2=20190930&range=1'.format(search=search)
        driver = webdriver.Chrome("./chromedriver.exe")
        driver.get(url_temp)
        time.sleep(2)
        return dongaUrlList(driver)

In [2]:
# 2단계 목록에서 href 가져오기
def dongaUrlList(driver):
    url_temp = driver.current_url 
    # 630까지임.
    for pageNo in range(1, 650, 15):
        url = url_temp + "&p={pageNo}".format(pageNo=pageNo)
        driver.get(url)
        time.sleep(2)
        dongaUrls_List = []
        dongaUrls = driver.find_elements_by_css_selector(".searchList .tit a")
        if len(dongaUrls) == 0:
            break
        for urlList in dongaUrls:
            if urlList.get_attribute("href")[8:11] == 'www':
                dongaUrls_List.append(urlList.get_attribute("href"))
            else:
                pass
        dongaInfo(driver, dongaUrls_List)

In [5]:
def dongaInfo(driver, dongaUrls_List):
    for urllist in dongaUrls_List:
        driver.get(urllist)
        time.sleep(2)
#         . 은 class, # 은 id값, HTML tag는 그대로 표현됨
    
        try:
            # 제목
            newsTitleList = driver.find_elements_by_css_selector(".article_title .title")
            newsTitle_temp = newsTitleList[0]
            newsTitle = newsTitle_temp.text
            
            # 부제목
            newsSubtitle = ""
            if driver.find_elements_by_css_selector(".sub_title") != []:
                newsSubtitleList = driver.find_elements_by_css_selector(".sub_title")
                newsSubtitle_temp = newsSubtitleList[0]
                newsSubtitle = newsSubtitle_temp.text
            else:
                pass   
            
            # 날짜
            newsDate = ""
            newsDateList = driver.find_elements_by_css_selector(".title_foot .date01")
            newsDate_temp = newsDateList[0]
            newsDate = newsDate_temp.text
            newsDate = newsDate.replace("입력 ", "").replace("-", "")[0:8]

            # 내용
            newsContents = ""
            newsContentsList = driver.find_elements_by_css_selector(".article_txt")
            for newsContent in newsContentsList:
                newsContents = newsContents + newsContent.text

            #  print
            print("url: ", urllist)
            print("제목: ", newsTitle)
            print("소제목: ", newsSubtitle)
#             print("날짜: ", newsDate)
#             print("내용: ", newsContents)       
            
        except:
            print("-----에러 url-----", urllist)
            continue
        
        dbData = [[urllist, newsTitle, newsDate, newsSubtitle, newsContents]]
        connectDB(dbData)

In [7]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'final'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into donga(url, title, date, sub_title, contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [12]:
# search = input("검색: ")
search = '일본 여행'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
dongaUrl(search)

url:  https://www.donga.com/news/article/all/20190424/95212926/4
제목:  [김재범 기자의 투얼로지] 보는 것 넘어 느끼는 여행…“실버세대도 반했죠”
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95212927/4
제목:  3월 방한관광객 전년대비 12.4% 증가
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95204616/1
제목:  이즈미 더블 스핀 면도기, 내 남자를 위한 생활 밀착형 선물
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95209759/1
제목:  LF 바네사브루노, ‘카바스 보야지’ 팝업스토어 오픈
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95206179/2
제목:  이번 주말 광주, 글로벌 한류팬 1만명 모인다
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95205991/1
제목:  ‘성접대·횡령’ 승리 신병처리 임박…금주 후반 영장 검토
소제목:  ‘유리홀딩스 자금 수천만원 횡령’으로 영장신청 검토
어제 유인석과 장시간 조사…불법촬영 등 4개 혐의
url:  https://www.donga.com/news/article/all/20190424/95202813/2
제목:  승리 300명 초대, 청담동 고급 주점 전세 내 파티…성매매 알선 의심
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95202027/2
제목:  “외국인의 시선으로 본 한국여행의 매력 소개합니다”
소제목:  
url:  https://www.donga.com/news/article/all/20190424/95200925/2
제목:  日만

url:  https://www.donga.com/news/article/all/20190402/94849257/1
제목:  해외 렌터카 여행, 어디서 가장 많이 할까
소제목:  최근 FIT 급부상으로 렌터카 여행 인기
가장 많이 이용하는 지역 1위는 하와이
url:  https://www.donga.com/news/article/all/20190402/94847151/1
제목:  여행사로 해외여행 덜 간다…자유여행 수요 32.7%↓
소제목:  주요 대형여행사, 3월 해외여행 수요 증가율 발표
지역별 비중은 일본 줄고 베트남 늘었다
url:  https://www.donga.com/news/article/all/20190401/94824802/1
제목:  책 모서리 접고… 펼친 채 사진 찍고… 파는 책을 내 책처럼 함부로
소제목:  [컬처 까talk]에티켓 실종된 쉼터형 서점
url:  https://www.donga.com/news/article/all/20190329/94800776/1
제목:  대구 부산 전주 찍고… 들불처럼 타오른 “대한독립만세”
소제목:  [3·1운동 100주년 현장/창간 99주년]
